In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install thop
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import models, transforms
from torchvision.models import DenseNet121_Weights, EfficientNet_B2_Weights

import os
import numpy as np
import matplotlib.pyplot as plt
import time
import json
import glob
import copy
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tqdm import tqdm
from thop import profile # For FLOPs calculation


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [3]:
# --- Hyperparameters and Configuration ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")
LEARNING_RATE = 0.001
BATCH_SIZE = 32
NUM_EPOCHS = 40 # Increased epochs for better tuning
IMG_SIZE = 224 
PATIENCE = 7 # Increased patience

# Data paths
TRAIN_REAL_DIR = train_real_dir
TRAIN_FAKE_DIR = train_fake_dir
TEST_DIR = test_dir

# Model save paths
DENSENET_PATH = "best_densenet.pth"
EFFICIENTNET_B2_PATH = "best_efficientnet_b2.pth"
COMPLEX_CNN_V2_PATH = "best_complex_cnn_v2.pth"
MODERN_CNN_PATH = "best_modern_cnn.pth"

Using device: cuda


NameError: name 'train_real_dir' is not defined

In [ ]:
class DeepfakeTrainDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None):
        self.transform = transform
        self.image_files = []
        for filename in glob.glob(os.path.join(real_dir, "*.png")):
            self.image_files.append((filename, 0.0))
        for filename in glob.glob(os.path.join(fake_dir, "*.png")):
            self.image_files.append((filename, 1.0))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path, label = self.image_files[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            image = Image.new("RGB", (IMG_SIZE, IMG_SIZE), color="black")
            label = 0.0
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.float32)

class DeepfakeTestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.transform = transform
        self.image_files = glob.glob(os.path.join(test_dir, "*.png"))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        filename = os.path.basename(img_path)
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading test image {img_path}: {e}")
            image = Image.new("RGB", (IMG_SIZE, IMG_SIZE), color="black")
        if self.transform:
            image = self.transform(image)
        return image, filename

# --- Data Transforms (Copied from V1) ---
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    normalize
])
val_test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    normalize
])

# --- Create DataLoaders (Copied from V1) ---
full_dataset = DeepfakeTrainDataset(TRAIN_REAL_DIR, TRAIN_FAKE_DIR, transform=train_transforms)
total_size = len(full_dataset)
val_size = int(total_size * 0.2)
train_size = total_size - val_size

if train_size > 0 and val_size > 0:
    train_dataset, val_dataset_with_aug = random_split(full_dataset, [train_size, val_size])
    val_dataset = DeepfakeTrainDataset(TRAIN_REAL_DIR, TRAIN_FAKE_DIR, transform=val_test_transforms)
    val_dataset.image_files = [full_dataset.image_files[i] for i in val_dataset_with_aug.indices]
else:
    print("Dataset too small to split. Using full dataset for both training and validation.")
    train_dataset = full_dataset
    val_dataset = DeepfakeTrainDataset(TRAIN_REAL_DIR, TRAIN_FAKE_DIR, transform=val_test_transforms)

print(f"Total training images: {len(train_dataset)}")
print(f"Total validation images: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_dataset = DeepfakeTestDataset(TEST_DIR, transform=val_test_transforms)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
print(f"Test images: {len(test_dataset)}")

In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, patience, model_save_path):
    """
    Main training and validation loop with early stopping.
    """
    
    if DEVICE == "cuda":
        torch.cuda.reset_peak_memory_stats(DEVICE)
        
    start_time = time.time()
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'val_precision': [], 'val_recall': [], 'val_f1': []}
    
    best_model_state = copy.deepcopy(model.state_dict())
    best_val_loss = float('inf')
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # --- Training Phase ---
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0
        train_total = 0

        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE).unsqueeze(1) # Match shape for BCEWithLogitsLoss [batch_size, 1]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            preds = torch.sigmoid(outputs) > 0.5
            running_corrects += torch.sum(preds == (labels > 0.5))
            train_total += labels.size(0)

        epoch_train_loss = running_loss / train_total
        epoch_train_acc = running_corrects.double() / train_total
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc.item())
        print(f'Train Loss: {epoch_train_loss:.4f} Acc: {epoch_train_acc:.4f}')

        # --- Validation Phase ---
        model.eval()   # Set model to evaluate mode
        running_val_loss = 0.0
        all_labels = []
        all_preds = []

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validation"):
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE).unsqueeze(1)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item() * inputs.size(0)
                preds = torch.sigmoid(outputs) > 0.5
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())
        
        if len(all_labels) > 0:
            val_total = len(all_labels)
            epoch_val_loss = running_val_loss / val_total
            all_labels = np.array(all_labels).flatten()
            all_preds = np.array(all_preds).flatten()
            epoch_val_acc = accuracy_score(all_labels, all_preds)
            epoch_val_precision = precision_score(all_labels, all_preds, zero_division=0)
            epoch_val_recall = recall_score(all_labels, all_preds, zero_division=0)
            epoch_val_f1 = f1_score(all_labels, all_preds, zero_division=0)
        else:
            print("Validation set is empty. Skipping metrics.")
            epoch_val_loss, epoch_val_acc, epoch_val_precision, epoch_val_recall, epoch_val_f1 = 0.0, 0.0, 0.0, 0.0, 0.0
            
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc)
        history['val_precision'].append(epoch_val_precision)
        history['val_recall'].append(epoch_val_recall)
        history['val_f1'].append(epoch_val_f1)

        print(f'Val Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.4f}')
        print(f'Val Precision: {epoch_val_precision:.4f} Recall: {epoch_val_recall:.4f} F1: {epoch_val_f1:.4f}')

        # --- Early Stopping Check ---
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_model_state = copy.deepcopy(model.state_dict())
            torch.save(best_model_state, model_save_path)
            model_size_mb = os.path.getsize(model_save_path) / (1024 * 1024)
            print(f"New best model saved to {model_save_path} (Size: {model_size_mb:.2f} MB)")
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            print(f"Early stopping counter: {early_stopping_counter}/{patience}")
        
        if early_stopping_counter >= patience:
            print("Early stopping triggered")
            break

    end_time = time.time()
    total_duration_sec = end_time - start_time
    print(f'\nTraining complete in {total_duration_sec // 60:.0f}m {total_duration_sec % 60:.0f}s')
    print(f'Best val Loss: {best_val_loss:4f}')
    
    if DEVICE == "cuda":
        peak_mem_mb = torch.cuda.max_memory_allocated(DEVICE) / (1024 * 1024)
        print(f'Peak GPU memory allocated during training: {peak_mem_mb:.2f} MB')
    
    model.load_state_dict(torch.load(model_save_path))
    return history, total_duration_sec

def plot_history(history, title):
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.legend()
    plt.title(f'{title} - Loss')
    plt.subplot(2, 2, 2)
    plt.plot(history['train_acc'], label='Train Acc')
    plt.plot(history['val_acc'], label='Val Acc')
    plt.legend()
    plt.title(f'{title} - Accuracy')
    plt.subplot(2, 2, 3)
    plt.plot(history['val_precision'], label='Val Precision')
    plt.plot(history['val_recall'], label='Val Recall')
    plt.legend()
    plt.title(f'{title} - Precision & Recall')
    plt.subplot(2, 2, 4)
    plt.plot(history['val_f1'], label='Val F1-Score')
    plt.legend()
    plt.title(f'{title} - F1-Score')
    plt.tight_layout()
    plt.show()

In [ ]:
print("--- Training Model 1: DenseNet-121 ---")

# 1. Load pre-trained model
model_densenet = models.densenet121(weights=DenseNet121_Weights.DEFAULT)

# 2. Freeze all parameters
for param in model_densenet.parameters():
    param.requires_grad = False

# 3. Replace the final layer
in_features = model_densenet.classifier.in_features
model_densenet.classifier = nn.Linear(in_features, 1)

# 4. Move model to device
model_densenet = model_densenet.to(DEVICE)

# 5. Calculate FLOPs and Params
try:
    dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(DEVICE)
    flops, params = profile(model_densenet, inputs=(dummy_input, ), verbose=False)
    print(f"DenseNet-121: {flops/1e9:.2f} GFLOPs, {params/1e6:.2f} MParams")
except Exception as e:
    print(f"Could not calculate FLOPs for DenseNet-121: {e}")
    
# 6. Define Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer_densenet = optim.Adam(model_densenet.classifier.parameters(), lr=LEARNING_RATE)

# 7. Train the model
densenet_history, densenet_time = train_model(model_densenet, criterion, optimizer_densenet, train_loader, val_loader, 
                                            num_epochs=NUM_EPOCHS, patience=PATIENCE, model_save_path=DENSENET_PATH)

# 8. Plot history
plot_history(densenet_history, "DenseNet-121 Fine-tuning")

In [ ]:
print("--- Training Model 2: EfficientNet-B2 (Unfrozen) ---")

# 1. Load pre-trained model
model_efficientnet_b2 = models.efficientnet_b2(weights=EfficientNet_B2_Weights.DEFAULT)

# 2. Freeze all parameters INITIALLY
for param in model_efficientnet_b2.parameters():
    param.requires_grad = False

# 3. Replace the final layer (which will be trainable by default)
in_features = model_efficientnet_b2.classifier[1].in_features
model_efficientnet_b2.classifier[1] = nn.Linear(in_features, 1)

# 4. Unfreeze the last two feature blocks
print("Unfreezing last two blocks of EfficientNet-B2...")
for param in model_efficientnet_b2.features[-1].parameters():
    param.requires_grad = True
for param in model_efficientnet_b2.features[-2].parameters():
    param.requires_grad = True

# 5. Collect parameters that need gradients
params_to_update = []
print("Parameters to be trained:")
for name, param in model_efficientnet_b2.named_parameters():
    if param.requires_grad:
        params_to_update.append(param)
        print(f"\t{name}")

# 6. Move model to device
model_efficientnet_b2 = model_efficientnet_b2.to(DEVICE)

# 7. Calculate FLOPs and Params
try:
    dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(DEVICE)
    flops, params = profile(model_efficientnet_b2, inputs=(dummy_input, ), verbose=False)
    print(f"EfficientNet-B2: {flops/1e9:.2f} GFLOPs, {params/1e6:.2f} MParams")
except Exception as e:
    print(f"Could not calculate FLOPs for EfficientNet-B2: {e}")

# 8. Define Loss and Optimizer
# We only optimize the parameters we explicitly unfroze
criterion = nn.BCEWithLogitsLoss()
optimizer_efficientnet_b2 = optim.Adam(params_to_update, lr=LEARNING_RATE)

# 9. Train the model
efficientnet_b2_history, efficientnet_b2_time = train_model(model_efficientnet_b2, criterion, optimizer_efficientnet_b2, train_loader, val_loader, 
                                                         num_epochs=NUM_EPOCHS, patience=PATIENCE, model_save_path=EFFICIENTNET_B2_PATH)

# 10. Plot history
plot_history(efficientnet_b2_history, "EfficientNet-B2 (Unfrozen) Tuning")

In [ ]:
class ComplexCNN_v2(nn.Module):
    def __init__(self):
        super(ComplexCNN_v2, self).__init__()
        
        # Input: [B, 3, 224, 224]
        self.conv1 = self._conv_block(3, 16, 3, 1, 1) # -> [B, 16, 112, 112]
        self.conv2 = self._conv_block(16, 32, 3, 1, 1) # -> [B, 32, 56, 56]
        self.conv3 = self._conv_block(32, 64, 3, 1, 1) # -> [B, 64, 28, 28]
        self.conv4 = self._conv_block(64, 128, 3, 1, 1) # -> [B, 128, 14, 14]
        
        # --- THIS IS THE FIX ---
        # Global Average Pooling: Takes [B, 128, 14, 14] -> [B, 128, 1, 1]
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # New FC stack with far fewer parameters
        self.fc_stack = nn.Sequential(
            nn.Flatten(), # -> [B, 128]
            nn.Linear(128, 512), # 128 * 512 = ~65k params (vs 12.9M!)
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1) # 512 * 1 = ~512 params
        )
        # --- END FIX ---

    def _conv_block(self, in_channels, out_channels, kernel, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel, stride=stride, padding=padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool(x) # Apply GAP
        x = self.fc_stack(x) # Apply new FC stack
        return x

In [ ]:
print("--- Training Model 3: ComplexCNN_v2 ---")

# 1. Instantiate model
model_complex_cnn_v2 = ComplexCNN_v2().to(DEVICE)

# 2. Calculate FLOPs and Params
try:
    dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(DEVICE)
    flops, params = profile(model_complex_cnn_v2, inputs=(dummy_input, ), verbose=False)
    print(f"ComplexCNN_v2: {flops/1e9:.2f} GFLOPs, {params/1e6:.2f} MParams")
    print("Note: Params should be dramatically lower than V1's 12.94M")
except Exception as e:
    print(f"Could not calculate FLOPs for ComplexCNN_v2: {e}")

# 3. Define Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer_complex_cnn_v2 = optim.Adam(model_complex_cnn_v2.parameters(), lr=LEARNING_RATE)

# 4. Train the model
complex_cnn_v2_history, complex_cnn_v2_time = train_model(model_complex_cnn_v2, criterion, optimizer_complex_cnn_v2, train_loader, val_loader, 
                                                         num_epochs=NUM_EPOCHS, patience=PATIENCE, model_save_path=COMPLEX_CNN_V2_PATH)

# 5. Plot history
plot_history(complex_cnn_v2_history, "ComplexCNN_v2 Training")

In [ ]:
class ModernCNN(nn.Module):
    def __init__(self):
        super(ModernCNN, self).__init__()
        
        self.features = nn.Sequential(
            # Block 1: [B, 3, 224, 224] -> [B, 32, 112, 112]
            self._vgg_block(3, 32, 2),
            # Block 2: -> [B, 64, 56, 56]
            self._vgg_block(32, 64, 2),
            # Block 3: -> [B, 128, 28, 28]
            self._vgg_block(64, 128, 2),
            # Block 4: -> [B, 256, 14, 14]
            self._vgg_block(128, 256, 2),
            # Block 5: -> [B, 512, 7, 7]
            self._vgg_block(256, 512, 2)
        )
        
        # Global Average Pooling: [B, 512, 7, 7] -> [B, 512, 1, 1]
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(), # -> [B, 512]
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1)
        )

    def _vgg_block(self, in_channels, out_channels, num_convs):
        layers = []
        for _ in range(num_convs):
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False))
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.GELU())
            in_channels = out_channels
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x


In [ ]:
print("--- Training Model 4: ModernCNN ---")

# 1. Instantiate model
model_modern_cnn = ModernCNN().to(DEVICE)

# 2. Calculate FLOPs and Params
try:
    dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(DEVICE)
    flops, params = profile(model_modern_cnn, inputs=(dummy_input, ), verbose=False)
    print(f"ModernCNN: {flops/1e9:.2f} GFLOPs, {params/1e6:.2f} MParams")
except Exception as e:
    print(f"Could not calculate FLOPs for ModernCNN: {e}")

# 3. Define Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer_modern_cnn = optim.Adam(model_modern_cnn.parameters(), lr=LEARNING_RATE)

# 4. Train the model
modern_cnn_history, modern_cnn_time = train_model(model_modern_cnn, criterion, optimizer_modern_cnn, train_loader, val_loader, 
                                                  num_epochs=NUM_EPOCHS, patience=PATIENCE, model_save_path=MODERN_CNN_PATH)

# 5. Plot history
plot_history(modern_cnn_history, "ModernCNN Training")

In [ ]:
def generate_predictions(model, loader, device):
    """Runs inference on the test set and returns a prediction dictionary."""
    model.eval() 
    predictions = {}
    with torch.no_grad():
        for inputs, filenames in tqdm(loader, desc="Generating Predictions"):
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = torch.sigmoid(outputs).cpu().numpy().flatten()
            for filename, prob in zip(filenames, probs):
                predictions[filename] = float(prob)
    return predictions


In [ ]:
print("--- Generating Final Predictions (V2 Models) ---")

# --- DenseNet-121 Predictions ---
try:
    print("Loading best DenseNet-121 model...")
    model_densenet.load_state_dict(torch.load(DENSENET_PATH))
    preds = generate_predictions(model_densenet, test_loader, DEVICE)
    with open("densenet_predictions.json", "w") as f:
        json.dump(preds, f, indent=4)
    print("DenseNet-121 predictions saved.")
except (FileNotFoundError, NameError):
    print("DenseNet model file not found. Skipping.")

# --- EfficientNet-B2 Predictions ---
try:
    print("Loading best EfficientNet-B2 model...")
    model_efficientnet_b2.load_state_dict(torch.load(EFFICIENTNET_B2_PATH))
    preds = generate_predictions(model_efficientnet_b2, test_loader, DEVICE)
    with open("efficientnet_b2_predictions.json", "w") as f:
        json.dump(preds, f, indent=4)
    print("EfficientNet-B2 predictions saved.")
except (FileNotFoundError, NameError):
    print("EfficientNet-B2 model file not found. Skipping.")

# --- ComplexCNN_v2 Predictions ---
try:
    print("Loading best ComplexCNN_v2 model...")
    model_complex_cnn_v2.load_state_dict(torch.load(COMPLEX_CNN_V2_PATH))
    preds = generate_predictions(model_complex_cnn_v2, test_loader, DEVICE)
    with open("complex_cnn_v2_predictions.json", "w") as f:
        json.dump(preds, f, indent=4)
    print("ComplexCNN_v2 predictions saved.")
except (FileNotFoundError, NameError):
    print("ComplexCNN_v2 model file not found. Skipping.")

# --- ModernCNN Predictions ---
try:
    print("Loading best ModernCNN model...")
    model_modern_cnn.load_state_dict(torch.load(MODERN_CNN_PATH))
    preds = generate_predictions(model_modern_cnn, test_loader, DEVICE)
    with open("modern_cnn_predictions.json", "w") as f:
        json.dump(preds, f, indent=4)
    print("ModernCNN predictions saved.")
except (FileNotFoundError, NameError):
    print("ModernCNN model file not found. Skipping.")


In [ ]:
# Clean up GPU memory
if DEVICE == "cuda":
    torch.cuda.empty_cache()